<programa> ::= <declaracoes> <atribuicoes>

<declaracoes> ::= <declaracao> |
                   <declaracao> <declaracoes>

<declaracao> ::= variavel texto |
                  variavel inteiro

<atribuicoes> ::= <atribuicao> |
                   <atribuicao> <atribuicoes>

<atribuicao> ::= variavel = <expressao>

<expressao> ::= variavel |
                 numero |
                 <expressao> mais <expressao>
                 <expressao> menos <expressao>
                 <expressao> dividido_por <expressao>
                 <expressao> multiplicado_por <expressao>

<se-senao> ::= se <expressao> <comparador> <expressao> faz <atribuicao> ; |
                se <expressao> <comparador> <expressao> faz <se-senao>; |
                se <expressao> <comparador> <expressao> faz <atribuicao> caso_contrario_faz <atribuicao>; |
                se <expressao> <comparador> <expressao> faz <atribuicao> caso_contrario_faz <se-senao>; |
                se <expressao> <comparador> <expressao> faz <se-senao> caso_contrario_faz <atribuicao>; |
                se <expressao> <comparador> <expressao> faz <se-senao> caso_contrario_faz <se-senao>; |


In [ ]:
#imports
from rply import LexerGenerator

In [ ]:
lg = LexerGenerator()

lg.add('NUMERO', r'\d+')
lg.add('MAIS', r'mais')
lg.add('MENOS', r'menos')
lg.add('MULTIPLICACAO', r'multiplicado_por')
lg.add('DIVISAO', r'dividido_por')


lg.add('INTEIRO', r'inteiro')
lg.add('TEXTO', r'texto')
lg.add('PONTOVIRGULA', r'\;')
lg.add('VARIAVEL', r'[a-zA-z][a-zA-z0-9]*')
lg.add('IGUAL', r'=')
#adicionar if


lg.ignore('\s+')
lexer = lg.build()